In [1]:
import pandas as pd
import re
import numpy as np

In [29]:
poems = pd.read_parquet('datasets/poems-with-targets-04-02-2022.parquet')


In [30]:
poems.head(1)

,title,poem,poet,tags,nationality,dates,sex,number_of_lines,number_of_stanzas
0,\r\r\n Objects Used to Prop Open a Window\r\r\n,"\r\r\nDog bone, stapler,\r\r\ncribbage board, garlic press\r\r\n because this window is loose—lacks\r\r\nsuction, lacks grip.\r\r\nBungee cord, bootstrap,\r\r\ndog leash, leather belt\r\r\n because this window had sash cords.\r\r\nThey frayed. They broke.\r\r\nFeather duster, thatch of straw, empty\r\r\nbottle of Elmer's glue\r\r\n because this window is loud—its hinges clack\r\r\nopen, clack shut.\r\r\nStuffed bear, baby blanket,\r\r\nsingle crib newel\r\r\n because this window is split. It's dividing\r\r\nin two.\r\r\nVelvet moss, sagebrush,\r\r\nwillow branch, robin's wing\r\r\n because this window, it's pane-less. It's only\r\r\na frame of air.\r\r\n",Michelle Menting,None,None,NaN,1.0,20,1


In [31]:
poems['title'] = poems.title.apply(lambda x: re.sub(r'\n','',re.sub(r'\r','',x)).strip()).copy()

In [32]:
poems['poem'] = poems.poem.apply(lambda x: re.sub(r'\n',' ',re.sub(r'\r','',x))).copy()

In [33]:
poems.head(1)

,title,poem,poet,tags,nationality,dates,sex,number_of_lines,number_of_stanzas
0,Objects Used to Prop Open a Window,"Dog bone, stapler, cribbage board, garlic press because this window is loose—lacks suction, lacks grip. Bungee cord, bootstrap, dog leash, leather belt because this window had sash cords. They frayed. They broke. Feather duster, thatch of straw, empty bottle of Elmer's glue because this window is loud—its hinges clack open, clack shut. Stuffed bear, baby blanket, single crib newel because this window is split. It's dividing in two. Velvet moss, sagebrush, willow branch, robin's wing because this window, it's pane-less. It's only a frame of air.",Michelle Menting,None,None,NaN,1.0,20,1


In [34]:
len(poems.tags.unique())

8285

In [35]:
all_tags = []

for tags in poems.tags.unique():
    try:
        for tag in tags.split(','):
            if tag.strip() in ['Seas','Rivers','& Streams']:
                all_tags.append('Seas, Rivers & Streams')
            else:
                all_tags.append(tag.strip())
    except:
        pass

In [36]:
all_tags = list(set(all_tags))
all_tags

['Sorrow & Grieving',
 'War & Conflict',
 'Easter',
 'Town & Country Life',
 'Disappointment & Failure',
 "St. Patrick's Day",
 'Marriage & Companionship',
 'Break-ups & Vexed Love',
 'Philosophy',
 'Pets',
 'Eating & Drinking',
 'Buddhism',
 'Engagement',
 'Cinco de Mayo',
 'Family & Ancestors',
 'Kwanzaa',
 'Unrequited Love',
 'Cities & Urban Life',
 'Death',
 'Rosh Hashanah',
 'Ghosts & the Supernatural',
 'Jobs & Working',
 'Relationships',
 'Fall',
 'Get Well & Recovery',
 'Coming of Age',
 'The Body',
 'Youth',
 "Mother's Day",
 'Time & Brevity',
 'Lesbian',
 'Memorial Day',
 'Nature',
 'Parenthood',
 'Trees & Flowers',
 'Passover',
 'Heroes & Patriotism',
 'Midlife',
 'Social Commentaries',
 'Home Life',
 'Men & Women',
 'Class',
 'Race & Ethnicity',
 'God & the Divine',
 'Planets',
 'Realistic & Complicated',
 'Theater & Dance',
 'Arts & Sciences',
 'Judaism',
 'Islam',
 'Seas, Rivers & Streams',
 'History & Politics',
 'School & Learning',
 'Animals',
 'Weddings',
 'Infatuatio

In [37]:
def check_tags(tags):
    try:
        tag_check = {'Seas, Rivers & Streams' if tag.strip() in ['Seas','Rivers','& Streams'] else tag.strip(): 1 for tag in tags.split(',')}
        all_tag_markup = {tag: tag_check[tag] if tag in tag_check.keys() else 0 for tag in all_tags}
        return all_tag_markup
    except:
        all_tag_markup = {tag: np.nan for tag in all_tags}
        return all_tag_markup

In [38]:
pd.DataFrame(list(poems['tags'].apply(check_tags).values))

,Sorrow & Grieving,War & Conflict,Easter,Town & Country Life,Disappointment & Failure,St. Patrick's Day,Marriage & Companionship,Break-ups & Vexed Love,Philosophy,Pets,...,Birthdays,Ramadan,Christianity,The Spiritual,Language & Linguistics,Desire,Valentine's Day,Weather,Separation & Divorce,Living
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13754,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13756,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
poems = pd.concat([poems.reset_index(drop=True),pd.DataFrame(list(poems['tags'].apply(check_tags).values))],axis=1).drop('tags',axis=1)

In [11]:
poems.head(1)

,title,poem,poet,nationality,dates,sex,number_of_lines,number_of_stanzas,sorrow_and_grieving,war_and_conflict,...,birthdays,ramadan,christianity,the_spiritual,language_and_linguistics,desire,valentine's_day,weather,separation_and_divorce,living
0,Objects Used to Prop Open a Window,"Dog bone, stapler, cribbage board, garlic press because this window is loose—lacks suction, lacks grip. Bungee cord, bootstrap, dog leash, leather belt because this window had sash cords. They frayed. They broke. Feather duster, thatch of straw, empty bottle of Elmer's glue because this window is loud—its hinges clack open, clack shut. Stuffed bear, baby blanket, single crib newel because this window is split. It's dividing in two. Velvet moss, sagebrush, willow branch, robin's wing because this window, it's pane-less. It's only a frame of air.",Michelle Menting,None,NaN,1.0,20,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The New Church,"The old cupola glinted above the clouds, shone among fir trees, but it took him an hour for the half mile all the way up the hill. As he trailed, the village passed him by, greeted him, asked about his health, but everybody hurried to catch the mass, left him leaning against fences, measuring the road with the walking stick he sculpted. He yearned for the day when the new church would be built—right across the road. Now it rises above the moon: saints in frescoes meet the eye, and only the rain has started to cut through the shingles on the roof of his empty house. The apple trees have taken over the sky, sequestered the gate, sidled over the porch.",Lucia Cherciu,None,NaN,1.0,14,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Look for Me,"Look for me under the hood of that old Chevrolet settled in weeds at the end of the pasture. I'm the radiator that spent its years bolted in front of an engine shoving me forward into the wind. Whatever was in me in those days has mostly leaked away, but my cap's still screwed on tight and I know the names of all these tattered moths and broken grasshoppers the rest of you've forgotten.",Ted Kooser,united_states,1939.0,0.0,12,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def snake_case(df):
    df.columns = [x.lower().replace(' ', '_').replace('&', 'and').replace(',','_') for x in df.columns]
    return df.columns

In [33]:
snake_case(poems)

Index(['title', 'poem', 'poet', 'nationality', 'dates', 'sex',
       'number_of_lines', 'number_of_stanzas', 'sorrow_and_grieving',
       'war_and_conflict',
       ...
       'birthdays', 'ramadan', 'christianity', 'the_spiritual',
       'language_and_linguistics', 'desire', 'valentine's_day', 'weather',
       'separation_and_divorce', 'living'],
      dtype='object', length=135)

### Check dtypes for target variables

- `sex`
- `country_of_origin`
- `dates`

In [7]:
targets = ['sex','country_of_origin','dates']

poems[targets].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13759 entries, 0 to 13758
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                13727 non-null  float64
 1   country_of_origin  8002 non-null   object 
 2   dates              7577 non-null   float64
dtypes: float64(2), object(1)
memory usage: 322.6+ KB


`sex` has been cast as a `float64` because there a np.nan values in this column. It needs to be converted to have male, female and None values. This will simplify EDA.

In [10]:
poems.sex = poems.sex.apply(lambda x: 'female' if x == 1.0 else 'male' if x == 0.0 else None).copy()

In [11]:
poems[targets].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13759 entries, 0 to 13758
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sex                13727 non-null  object 
 1   country_of_origin  8002 non-null   object 
 2   dates              7577 non-null   float64
dtypes: float64(1), object(2)
memory usage: 322.6+ KB


All target columns are the correct data type now. The final dataframe can be saved.

In [5]:
# poems = pd.read_parquet('datasets/poems-final.parquet')
poems.to_parquet('datasets/poems-final.parquet', index=False)

## Build Markdown Data Dictionary table

In [21]:
for index, column in enumerate(poems.columns):
    if index < 8: 
        print(f'|{column}|desc|{poems[column].dtypes}|{poems[column].isnull().sum()}|{len(poems[column]) - poems[column].isnull().sum()}|')
    else:
        print(f'|{column}|Indicates if `{column}` tag has been applied to poem. <n>1: `{column}` tag has been applied<\\n><n>0: `{column}` tag has not been applied.<\\n>|{poems[column].dtypes}|{poems[column].isnull().sum()}|{len(poems[column]) - poems[column].isnull().sum()}|') 

|title|desc|object|0|13759|
|poem|desc|object|0|13759|
|poet|desc|object|0|13759|
|country_of_origin|desc|object|5757|8002|
|dates|desc|float64|6182|7577|
|sex|desc|float64|32|13727|
|number_of_lines|desc|int64|0|13759|
|number_of_stanzas|desc|int64|0|13759|
|sorrow_and_grieving|Indicates if `sorrow_and_grieving` tag has been applied to poem. <n>1: `sorrow_and_grieving` tag has been applied<\n><n>0: `sorrow_and_grieving` tag has not been applied.<\n>|float64|902|12857|
|war_and_conflict|Indicates if `war_and_conflict` tag has been applied to poem. <n>1: `war_and_conflict` tag has been applied<\n><n>0: `war_and_conflict` tag has not been applied.<\n>|float64|902|12857|
|easter|Indicates if `easter` tag has been applied to poem. <n>1: `easter` tag has been applied<\n><n>0: `easter` tag has not been applied.<\n>|float64|902|12857|
|town_and_country_life|Indicates if `town_and_country_life` tag has been applied to poem. <n>1: `town_and_country_life` tag has been applied<\n><n>0: `town_and_